In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans

In [3]:
# optimized partitional models
clara_opt = pd.read_csv('../2_models/with_raw_data/wind/results/partitional_opt/csv_labels_raw/clara.csv')
fuzzy_opt = pd.read_csv('../2_models/with_raw_data/wind/results/partitional_opt/csv_labels_raw/fuzzy_c_means.csv')
kmeans_opt = pd.read_csv('../2_models/with_raw_data/wind/results/partitional_opt/csv_labels_raw/kmeans.csv')

# partitionalal models
clara = pd.read_csv('../2_models/with_raw_data/wind/results/partitional/csv_labels_raw/clara.csv')
fuzzy = pd.read_csv('../2_models/with_raw_data/wind/results/partitional/csv_labels_raw/fuzzy_c_means.csv')
kmeans = pd.read_csv('../2_models/with_raw_data/wind/results/partitional/csv_labels_raw/kmeans.csv')

# model based
bgmm = pd.read_csv('../2_models/with_raw_data/wind/results/model_based/csv_labels_cut_raw/bgmm.csv')
dpmm = pd.read_csv('../2_models/with_raw_data/wind/results/model_based/csv_labels_cut_raw/dpmm.csv')
gaussian_mix = pd.read_csv('../2_models/with_raw_data/wind/results/model_based/csv_labels_cut_raw/gaussian_mix.csv')

# optimized hierarchical models
average_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/average_link.csv')
birch_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/birch.csv')
single_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/single_link.csv')
centroid_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/centroid_link.csv')
ward_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/ward_link.csv')

# hierarchical models
average_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/average_link.csv')
birch = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/birch.csv')
single_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/single_link.csv')
centroid_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/centroid_link.csv')
ward_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/ward_link.csv')

# density based models
dbscan = pd.read_csv('../2_models/with_raw_data/wind/results/density_based/csv_labels_cut_raw/DBSCAN.csv')
optics = pd.read_csv('../2_models/with_raw_data/wind/results/density_based/csv_labels_cut_raw/OPTICS.csv')

In [4]:
# Define a function to compute evaluation metrics
def compute_metrics(df):
    metrics = {}
    
    # Silhouette Score
    if df['cluster_label'].nunique() > 1:
        metrics['Silhouette Score'] = silhouette_score(df[['PC1', 'PC2']], df['cluster_label'])
        # Davies-Bouldin Index
        metrics['Davies-Bouldin Index'] = davies_bouldin_score(df[['PC1', 'PC2']], df['cluster_label'])
        # Calinski-Harabasz Index
        metrics['Calinski-Harabasz Index'] = calinski_harabasz_score(df[['PC1', 'PC2']], df['cluster_label'])
        # Dunn Index
        min_intercluster_distances = []
        max_intracluster_distances = []
        for label in df['cluster_label'].unique():
            cluster_points = df[df['cluster_label'] == label][['PC1', 'PC2']]
            cluster_distances = pdist(cluster_points)
            num_clusters = min(len(df['cluster_label'].unique()), len(cluster_points))
            kmeans = KMeans(n_clusters=num_clusters)
            kmeans.fit(cluster_points)
            min_intercluster_distances.append(min([np.linalg.norm(cluster_points.values - centroid) for centroid in kmeans.cluster_centers_]))
            if len(cluster_distances) > 0:  # Check if cluster_distances has any elements
                max_intracluster_distances.append(max(cluster_distances))
            else:
                max_intracluster_distances.append(0)  # Append a default value
        metrics['Dunn Index'] = min(min_intercluster_distances) / max(max_intracluster_distances)
        
        # Gap Statistic
        def compute_gap_statistic(data, k_max=5, n_refs=5):
            def compute_reference_distribution(data, k):
                return np.mean([
                    np.log(pdist(KMeans(k).fit(data).cluster_centers_).mean())
                    for _ in range(n_refs)
                ])
            
            observed_log_wk = np.log(pdist(KMeans(k_max).fit(data).cluster_centers_).mean())
            reference_log_wk = compute_reference_distribution(data, k_max)
            gap = reference_log_wk - observed_log_wk
            
            return gap
        
        metrics['Gap Statistic'] = compute_gap_statistic(df[['PC1', 'PC2']])
        
        # Xie-Beni Index
        # Compute the total scatter
        total_scatter = np.sum(pdist(df[['PC1', 'PC2']]) ** 2) / (2 * len(df))
        
        # Compute the intra-cluster scatter
        cluster_labels = df['cluster_label'].unique()
        intra_cluster_scatter = 0
        for label in cluster_labels:
            cluster_points = df[df['cluster_label'] == label][['PC1', 'PC2']]
            cluster_center = np.mean(cluster_points, axis=0)
            intra_cluster_scatter += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1) ** 2)
        intra_cluster_scatter /= len(df)
        
        # Compute the Xie-Beni Index
        metrics['Xie-Beni Index'] = intra_cluster_scatter / total_scatter
    else:
        metrics['Silhouette Score'] = 0
        metrics['Davies-Bouldin Index'] = 0
        metrics['Calinski-Harabasz Index'] = 0
        metrics['Dunn Index'] = 0
        metrics['Gap Statistic'] = 0
        metrics['Xie-Beni Index'] = 0

    return metrics


In [5]:
# Create a dictionary with dataframes and their names
dataframes = {
    'clara_opt': clara_opt, 'fuzzy_opt': fuzzy_opt, 'kmeans_opt': kmeans_opt,
    'clara': clara, 'fuzzy': fuzzy, 'kmeans': kmeans,
    'bgmm': bgmm, 'dpmm': dpmm, 'gaussian_mix': gaussian_mix,
    'average_link_opt': average_link_opt, 'birch_opt': birch_opt,
    'single_link_opt': single_link_opt, 'centroid_link_opt': centroid_link_opt,
    'ward_link_opt': ward_link_opt,
    'average_link': average_link, 'birch': birch, 'single_link': single_link,
    'centroid_link': centroid_link, 'ward_link': ward_link,
    'dbscan': dbscan, 'optics': optics
 }

In [6]:
# Iterate over each dataframe and print the name of the ones without a 'cluster_label' column
for model_name, df in dataframes.items():
    if 'cluster_label' not in df.columns:
        print(f"The dataframe '{model_name}' does not have a column named 'cluster_label'.")
        print(df.columns)

The dataframe 'fuzzy_opt' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans_opt' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'clara' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster'], dtype='object')
The dataframe 'fuzzy' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')


In [7]:
# Loop through each dataframe and rename the 'cluster' column to 'cluster_label'
for model_name, df in dataframes.items():
    if 'cluster' in df.columns:
        df.rename(columns={'cluster': 'cluster_label'}, inplace=True)

In [8]:
# Initialize an empty list to store the results
results = []

# Compute and store the evaluation metrics for each dataframe
for model_name, df in dataframes.items():
    print(f'Evaluating {model_name}')
    metrics = compute_metrics(df)
    results.append({'Model': model_name, **metrics})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

Evaluating clara_opt


Evaluating fuzzy_opt
Evaluating kmeans_opt
Evaluating clara
Evaluating fuzzy
Evaluating kmeans
Evaluating bgmm
Evaluating dpmm
Evaluating gaussian_mix
Evaluating average_link_opt
Evaluating birch_opt
Evaluating single_link_opt
Evaluating centroid_link_opt
Evaluating ward_link_opt
Evaluating average_link
Evaluating birch
Evaluating single_link
Evaluating centroid_link
Evaluating ward_link
Evaluating dbscan
Evaluating optics


In [9]:
# Display the results
results_df

,Model,Silhouette Score,Davies-Bouldin Index,Calinski-Harabasz Index,Dunn Index,Gap Statistic,Xie-Beni Index
0,clara_opt,0.339105,1.159549,28729.357093,32.531436,0.002328,0.000025
1,fuzzy_opt,0.326231,1.195801,28199.799276,33.416227,-0.011250,0.000026
2,kmeans_opt,0.363694,0.904383,35502.061880,24.617870,-0.004480,0.000017
3,clara,0.256436,1.375009,21488.184219,19.231252,-0.000598,0.000021
4,fuzzy,0.363019,0.906468,35439.232612,24.935407,-0.024864,0.000017
5,kmeans,0.363694,0.904383,35502.061880,24.653263,0.001511,0.000017
6,bgmm,0.182444,0.921804,16653.729223,5.784390,-0.006398,0.000017
7,dpmm,0.047532,1.417619,13258.441517,0.088854,-0.025303,0.000006
8,gaussian_mix,0.332889,0.796661,33701.452941,7.994980,-0.006737,0.000005
9,average_link_opt,0.276272,0.904987,17425.286979,0.171031,-0.007326,0.000020


In [10]:
results_df.to_csv('results/raw_wind.csv', index=False)